# 2. Grouping customers together!
Now we will deal with clustering algorithms that will provide groups of clients which are similar among them.

To solve this task, you must accomplish the following stages:

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import random

In [165]:
data = pd.read_csv('bank_transactions.csv')
data.head(10)

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.00
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.00
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.00
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.00
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.50
5,T6,C1536588,8/10/72,F,ITANAGAR,53609.20,2/8/16,173940,676.00
6,T7,C7126560,26/1/92,F,MUMBAI,973.46,2/8/16,173806,566.00
7,T8,C1220223,27/1/82,M,MUMBAI,95075.54,2/8/16,170537,148.00
8,T9,C8536061,19/4/88,F,GURGAON,14906.96,2/8/16,192825,833.00
9,T10,C6638934,22/6/84,M,MUMBAI,4279.22,2/8/16,192446,289.11


In [166]:
data.shape

(1048567, 9)

In [167]:
# drop all missing values - they are not that many 
data.dropna(inplace=True)

In [168]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1041614 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1041614 non-null  object 
 1   CustomerID               1041614 non-null  object 
 2   CustomerDOB              1041614 non-null  object 
 3   CustGender               1041614 non-null  object 
 4   CustLocation             1041614 non-null  object 
 5   CustAccountBalance       1041614 non-null  float64
 6   TransactionDate          1041614 non-null  object 
 7   TransactionTime          1041614 non-null  int64  
 8   TransactionAmount (INR)  1041614 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 79.5+ MB


In [169]:
# handling dates
data.CustomerDOB = pd.to_datetime(data.CustomerDOB)
data.TransactionDate = pd.to_datetime(data.TransactionDate)

In [170]:
data.TransactionTime = data.TransactionTime.apply(lambda x: datetime.strptime(str(x).zfill(6), '%H%M%S').time())

In [171]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1041614 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   TransactionID            1041614 non-null  object        
 1   CustomerID               1041614 non-null  object        
 2   CustomerDOB              1041614 non-null  datetime64[ns]
 3   CustGender               1041614 non-null  object        
 4   CustLocation             1041614 non-null  object        
 5   CustAccountBalance       1041614 non-null  float64       
 6   TransactionDate          1041614 non-null  datetime64[ns]
 7   TransactionTime          1041614 non-null  object        
 8   TransactionAmount (INR)  1041614 non-null  float64       
dtypes: datetime64[ns](2), float64(2), object(5)
memory usage: 79.5+ MB


In [172]:
# shift the date of births with 100 to the left (for those with a DOB year > 2000)
data.loc[data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] = data.loc[data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] - pd.DateOffset(years = 100)

# drop the customers that have year = 1800
data.drop(data[data.CustomerDOB.dt.year == 1800].index, axis=0, inplace=True)

In [173]:
# save the pre-processed dataframe to a pickle file
data.to_pickle('data_prepr.pkl')

In [75]:
df1 = pd.read_pickle('data_prepr.pkl')
df1.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,14:32:07,25.0
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,14:18:58,27999.0
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,14:27:12,459.0
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,14:27:14,2060.0
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,18:11:56,1762.5


# 2.1 Getting your data + feature engineering
2.Sometimes, the features (variables, fields) are not given in a dataset but can be created from it. The previous step is known as feature engineering. For example, the original dataset has several transactions done by the same customer. Then, we suggest you to group data by the client (using CustomerId) and, based on it, create the following new features for each CustomerId:

a) Number of transactions

b) Number of transactions with a balance bigger than 100 USD

c) Average amount of the transactions

d) Average balance

e) Average difference between the balance and the transaction amount for each customer (this is mainly known in the banking world as utilisation).

f) Most common gender of the customer

h) Most frequent location of the customer

So, in the end, you should have for each CustomerID seven features.

3.Consider at least 20 additional features that can be generated for each CustomerId. Describe each of them and why you believe they will be helpful. Moreover, add it to the previous dataset (the one with seven features). In the end, you should have for each CustomerID at least 27 features (7 recommended + 20 suggested by you).

Hints for feature engineering:

- Instead of only using the average, you could use other functions such as minimum, maximum, percentiles, etc.
- Think of adding filters to your features. For instance, in "Number of transactions with balance bigger than 100 USD”, 
  the filter was "bigger than 100 USD". Can you think about other filters, even including other variables?
- Think of including the information given by the fields transaction date and time in your calculations.
- Think about getting information from a customer’s birthday, even comparing it to other fields.
- You could also calculate percentages in your features. Think of a plausible set of percentages worth to be calculated.

In [76]:
# add a new feature CustomerAge = the age of the customer today
df1['CustAge'] = (( pd.to_datetime('today') - df1['CustomerDOB'] ) / np.timedelta64(1, 'Y')).round(0)
df1['CustAge'] = df1['CustAge'].astype(int)

In [77]:
# categorize the customers based on age
# young Age <= 25
# working Age>25 and Age<65
# retired Age>=65

# function that categorize a customer based on the age
def age_class(age):
    if age<=25:
        return 'young'
    elif ((age>25) and (age<65)):
        return 'working'
    else:
        return 'retired'

df1['Age_Class'] = df1['CustAge'].apply(lambda row: age_class(row))

In [78]:
# add a new feature that indicates if the balance > 100 USD
df1['Balance>100'] = np.where(df1['CustAccountBalance'] > 100, 1, 0)

# add a new feature that indicates if the balance > 1000 USD
df1['Balance>1000'] = np.where(df1['CustAccountBalance'] > 1000, 1, 0)

In [79]:
# add a new feature that indicates if the transaction amount > 1000 USD
df1['Trans_Amnt>1000'] = np.where(df1['TransactionAmount (INR)'] > 1000, 1, 0)

In [80]:
# add a new feature Utilisation = difference between the balance and the transaction amount
df1['Utilisation'] = df1['CustAccountBalance'] - df1['TransactionAmount (INR)']

In [81]:
# create a variable that indicates the type of the customer based on the balance
# gold > 1,000,000 USD
# silver 
# bronze < 300,000 USD

def cust_type(balance):
    if balance > 1000000:
        return 'gold'
    elif balance < 300000:
        return 'bronze'
    else:
        return 'silver'

df1['Cust_Type'] = df1['CustAccountBalance'].apply(lambda row: cust_type(row))

In [82]:
# add a feature that indicates the month in whcih the person was born
df1['CustDOB_Month'] = df1['CustomerDOB'].apply(lambda row: row.month)

In [83]:
# feature that indicates the month when the transaction was made
df1['Trans_Month'] = df1['TransactionDate'].apply(lambda row: row.month)

# feature that indicates the day of the week when the transaction was made
df1['Trans_DayWeek'] = df1['TransactionDate'].apply(lambda row: row.dayofweek+1)

In [84]:
# function that checks if the day is on the weekend and returns 1 if it is, 0 oth.
def is_weekend(day):
    if (day == 6) or (day == 7):
        return 1
    return 0

# feature that indicates if the transaction was made on the weekend
df1['Trans_On_Wknd'] = df1['Trans_DayWeek'].apply(lambda row: is_weekend(row))

In [85]:
# function that converts the month to a season
def season(month):
    if ((month==12) or (month==1) or (month==2)):
        return 'winter'
    elif ((month==3) or (month==4) or (month==5)):
        return 'spring'
    elif ((month==6) or (month==7) or (month==8)):
        return 'summer'
    else:
        return 'autumn'
    
# feature that indicates the season when the transaction was made (based on the month)
df1['Trans_Season'] = df1['Trans_Month'].apply(lambda row: season(row))

In [86]:
# based on the time, we can see if the transaction was made in the morning [6-12), 
# during the day [12-17), in the evening [17-22), or during the night [22-6)

def trans_time(time):
    if (time.hour >= 6) and (time.hour < 12):
        return 'morning'
    elif (time.hour >= 12) and (time.hour < 17):
        return 'day'
    elif (time.hour >= 17) and (time.hour < 22):
        return 'evening'
    else:
        return 'night'

# feature that indicates when the transaction was made throughout the day
df1['Trans_MDEN'] = df1['TransactionTime'].apply(lambda row: trans_time(row))

In [87]:
df1.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustAge,...,Balance>1000,Trans_Amnt>1000,Utilisation,Cust_Type,CustDOB_Month,Trans_Month,Trans_DayWeek,Trans_On_Wknd,Trans_Season,Trans_MDEN
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,14:32:07,25.0,28,...,1,0,17794.05,bronze,10,2,1,0,winter,day
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,14:18:58,27999.0,66,...,1,1,-25728.31,bronze,4,2,1,0,winter,day
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,14:27:12,459.0,26,...,1,0,17415.44,bronze,11,2,1,0,winter,day
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,14:27:14,2060.0,49,...,1,1,864443.21,silver,9,2,1,0,winter,day
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,18:11:56,1762.5,35,...,1,1,4951.93,bronze,3,2,1,0,winter,evening


In [88]:
# add a new feature that indicates whether the trans. amount is very large,large, medium, small
# very large VR, >5000 USD
# large L, >1000 and <=5000 USD
# medium M, >100 and <=1000 USD
# small S, <=100 USD

def trans_amnt(amount):
    if amount > 5000:
        return 'VL'
    elif ((amount > 1000) and (amount <= 5000)):
        return 'L'
    elif ((amount > 100) and (amount <= 1000)):
        return 'M'
    else:
        return 'S'

df1['Trans_Amnt_Type'] = df1['TransactionAmount (INR)'].apply(lambda row: trans_amnt(row))

In [89]:
# there are transactions of amount > current account balance
# we consider those as rejected transactions

# create a new feature that indicates if the transaction is rejected (Acc.Balance < Trans.Amnt) 
# or accepted (Acc.Balance > Trans.Amnt)
df1['Trans_Acc_Rej'] = np.where(df1['Utilisation'] < 0, 'rejected', 'accepted')

In [90]:
# ratio cust.acc.balance / total balance in the bank
total_bal = sum(df1['CustAccountBalance'])   # total balance across all accounts in the bank
df1['Ratio_AccBal'] = df1['CustAccountBalance'].apply(lambda row: row/total_bal)

In [91]:
# total account balance across all accounts with gender = F
total_bal_F = np.sum(df1[df1['CustGender'] == 'F']['CustAccountBalance'])

# total account balance across all accounts with gender = M
total_bal_M = np.sum(df1[df1['CustGender'] == 'M']['CustAccountBalance'])

df1['Ratio_AccBal_FM'] = np.where(df1['CustGender'] == 'F',df1['CustAccountBalance']/total_bal_F,df1['CustAccountBalance']/total_bal_M)

In [92]:
# total transaction amount across all accounts with gender = F
total_tramnt_F = np.sum(df1[df1['CustGender'] == 'F']['TransactionAmount (INR)'])

# total transaction amount across all accounts with gender = M
total_tramnt_M = np.sum(df1[df1['CustGender'] == 'M']['TransactionAmount (INR)'])

df1['Ratio_TrAmnt_FM'] = np.where(df1['CustGender'] == 'F',df1['TransactionAmount (INR)']/total_tramnt_F,df1['TransactionAmount (INR)']/total_tramnt_M)

In [93]:
df1.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustAge,...,Trans_Month,Trans_DayWeek,Trans_On_Wknd,Trans_Season,Trans_MDEN,Trans_Amnt_Type,Trans_Acc_Rej,Ratio_AccBal,Ratio_AccBal_FM,Ratio_TrAmnt_FM
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,14:32:07,25.0,28,...,2,1,0,winter,day,S,accepted,1.705100e-07,6.436753e-07,5.730795e-08
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,14:18:58,27999.0,66,...,2,1,0,winter,day,VL,rejected,2.172817e-08,2.955814e-08,2.814347e-05
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,14:27:12,459.0,26,...,2,1,0,winter,day,M,accepted,1.710400e-07,6.456762e-07,1.052174e-06
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,14:27:14,2060.0,49,...,2,1,0,winter,day,L,accepted,8.291546e-06,3.130059e-05,4.722175e-06
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,18:11:56,1762.5,35,...,2,1,0,winter,evening,L,accepted,6.425020e-08,2.425445e-07,4.040211e-06


In [95]:
# group the data by the Customer ID
df2 = df1.groupby('CustomerID').agg(Num_Trans = ('TransactionID', "count"),
                                    Num_Trans_Bal_100 = ('Balance>100', "sum"),
                                    Num_Trans_Bal_1000 = ('Balance>1000', "sum"),
                                    Num_Trans_Amnt_1000 = ('Trans_Amnt>1000', "sum"),
                                    
                                    Avg_Amnt_Trans = ('TransactionAmount (INR)', "mean"),
                                    Max_Amnt_Trans = ('TransactionAmount (INR)', "max"),
                                    Min_Amnt_Trans = ('TransactionAmount (INR)', "min"),
                                    
                                    Avg_Bal = ('CustAccountBalance', "mean"),
                                    Max_Bal = ('CustAccountBalance', "max"),
                                    Min_Bal = ('CustAccountBalance', "min"),
                                    
                                    Avg_Util = ('Utilisation', "mean"),
                                    
                                    Acct_Age = ('CustAge','mean'),
                                    Acct_AgeCl = ('Age_Class', pd.Series.mode),
                                    
                                    Acct_Gender = ('CustGender', pd.Series.mode),
                                    Acct_Location = ('CustLocation', pd.Series.mode),
                                    
                                    Acct_CustType = ('Cust_Type', pd.Series.mode)).reset_index()

In [99]:
df2.head()

,CustomerID,Num_Trans,Num_Trans_Bal_100,Num_Trans_Bal_1000,Num_Trans_Amnt_1000,Avg_Amnt_Trans,Max_Amnt_Trans,Min_Amnt_Trans,Avg_Bal,Max_Bal,Min_Bal,Avg_Util,Acct_Age,Acct_AgeCl,Acct_Gender,Acct_Location,Acct_CustType
0,C1010011,2,2,2,1,2553.0,4750.0,356.0,76340.635,120180.54,32500.73,73787.635,35.0,working,"[F, M]","[NEW DELHI, NOIDA]",bronze
1,C1010012,1,1,1,1,1499.0,1499.0,1499.0,24204.490,24204.49,24204.49,22705.490,28.0,working,M,MUMBAI,bronze
2,C1010014,2,2,2,1,727.5,1205.0,250.0,100112.950,161848.76,38377.14,99385.450,34.5,working,"[F, M]",MUMBAI,bronze
3,C1010018,1,1,0,0,30.0,30.0,30.0,496.180,496.18,496.18,466.180,32.0,working,F,CHAMPARAN,bronze
4,C1010024,1,1,1,1,5000.0,5000.0,5000.0,87058.650,87058.65,87058.65,82058.650,57.0,working,M,KOLKATA,bronze


In [100]:
df3 = df2.copy()

In [101]:
# when there is tie, pd.Series.mode returns a list of the most common values 
# we decided to choose on random one of those values
df3['Acct_Gender'] = df3['Acct_Gender'].apply(lambda row: random.choice(row))

In [102]:
def convert_list(info):
    if isinstance(info,str) == True:
        l = []
        l.append(info)
        return l
    return info

In [103]:
# convert all Location values to list of strings(locations)
df3['Acct_Location'] = df3['Acct_Location'].apply(lambda row: convert_list(row))
df3.head()

,CustomerID,Num_Trans,Num_Trans_Bal_100,Num_Trans_Bal_1000,Num_Trans_Amnt_1000,Avg_Amnt_Trans,Max_Amnt_Trans,Min_Amnt_Trans,Avg_Bal,Max_Bal,Min_Bal,Avg_Util,Acct_Age,Acct_AgeCl,Acct_Gender,Acct_Location,Acct_CustType
0,C1010011,2,2,2,1,2553.0,4750.0,356.0,76340.635,120180.54,32500.73,73787.635,35.0,working,M,"[NEW DELHI, NOIDA]",bronze
1,C1010012,1,1,1,1,1499.0,1499.0,1499.0,24204.490,24204.49,24204.49,22705.490,28.0,working,M,[MUMBAI],bronze
2,C1010014,2,2,2,1,727.5,1205.0,250.0,100112.950,161848.76,38377.14,99385.450,34.5,working,M,[MUMBAI],bronze
3,C1010018,1,1,0,0,30.0,30.0,30.0,496.180,496.18,496.18,466.180,32.0,working,F,[CHAMPARAN],bronze
4,C1010024,1,1,1,1,5000.0,5000.0,5000.0,87058.650,87058.65,87058.65,82058.650,57.0,working,M,[KOLKATA],bronze


In [104]:
# when we have a tie, choose one of the values on random
df3['Acct_Location'] = df3['Acct_Location'].apply(lambda row: random.choice(row))

In [105]:
# convert Cust_Type to a list of strings(types)
df3['Acct_CustType'] = df3['Acct_CustType'].apply(lambda row: convert_list(row))

# when we have a tie, choose one of the values on random
df3['Acct_CustType'] = df3['Acct_CustType'].apply(lambda row: random.choice(row))

In [106]:
# convert Acct_AgeCl to a list of strings(types)
df3['Acct_AgeCl'] = df3['Acct_AgeCl'].apply(lambda row: convert_list(row))

# when we have a tie, choose one of the values on random
df3['Acct_AgeCl'] = df3['Acct_AgeCl'].apply(lambda row: random.choice(row))

In [107]:
df3.head()

,CustomerID,Num_Trans,Num_Trans_Bal_100,Num_Trans_Bal_1000,Num_Trans_Amnt_1000,Avg_Amnt_Trans,Max_Amnt_Trans,Min_Amnt_Trans,Avg_Bal,Max_Bal,Min_Bal,Avg_Util,Acct_Age,Acct_AgeCl,Acct_Gender,Acct_Location,Acct_CustType
0,C1010011,2,2,2,1,2553.0,4750.0,356.0,76340.635,120180.54,32500.73,73787.635,35.0,working,M,NOIDA,bronze
1,C1010012,1,1,1,1,1499.0,1499.0,1499.0,24204.490,24204.49,24204.49,22705.490,28.0,working,M,MUMBAI,bronze
2,C1010014,2,2,2,1,727.5,1205.0,250.0,100112.950,161848.76,38377.14,99385.450,34.5,working,M,MUMBAI,bronze
3,C1010018,1,1,0,0,30.0,30.0,30.0,496.180,496.18,496.18,466.180,32.0,working,F,CHAMPARAN,bronze
4,C1010024,1,1,1,1,5000.0,5000.0,5000.0,87058.650,87058.65,87058.65,82058.650,57.0,working,M,KOLKATA,bronze


In [108]:
# save to pickle file
df3.to_pickle('data_CustID.pkl')

In [110]:
# data for each CustomerID (1 row per CustomerID)
df3 = pd.read_pickle('data_CustID.pkl')

In [112]:
# merge the big dataframe with the one that has specific info for each unique Cust ID

In [113]:
df_big = pd.merge(df1, df3)

In [118]:
df_big.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustAge,...,Min_Amnt_Trans,Avg_Bal,Max_Bal,Min_Bal,Avg_Util,Acct_Age,Acct_AgeCl,Acct_Gender,Acct_Location,Acct_CustType
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,14:32:07,25.0,28,...,25.0,17819.050,17819.05,17819.05,17794.050,28.0,working,F,JAMSHEDPUR,bronze
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,14:18:58,27999.0,66,...,27999.0,2270.690,2270.69,2270.69,-25728.310,66.0,retired,M,JHAJJAR,bronze
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,14:27:12,459.0,26,...,459.0,17874.440,17874.44,17874.44,17415.440,26.0,working,F,MUMBAI,bronze
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,14:27:14,2060.0,49,...,878.4,434849.495,866503.21,3195.78,433380.295,46.0,working,F,MUMBAI,bronze
4,T391139,C5342380,1979-05-29,M,MUMBAI,3195.78,2016-08-13,19:41:48,878.4,43,...,878.4,434849.495,866503.21,3195.78,433380.295,46.0,working,F,MUMBAI,bronze


In [119]:
# save to pickle file
df_big.to_pickle('data_big.pkl')

In [120]:
# data for each CustomerID (1 row per CustomerID)
df_big = pd.read_pickle('data_big.pkl')